In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler
from scipy.stats import skew



#Display optios
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)



In [91]:
#load the data 
train_df = pd.read_csv("train.csv") 
test_df  = pd.read_csv("test.csv")    

print(train_df.shape, test_df.shape)
display(train_df.head(10))  


(1460, 81) (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,

In [92]:
# remember to add visuals for the report
train_df = train_df.drop(
    train_df[(train_df['GrLivArea'] > 4000) & (train_df['SalePrice'] < 300000)].index
)


In [93]:
#Combine both the Train and Test set

train_df['DatasetType'] = 'train'
test_df['DatasetType']  = 'test'

full_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
print('Combine shape:', full_df.shape)

# copy of sales price in the train dataset 
target = full_df.loc[full_df['DatasetType'] == 'train', 'SalePrice'].copy()

Combine shape: (2917, 82)


In [97]:
#dropping the ID column becasue its not realevant yet

if "Id" in full_df.columns:
    full_df.drop(columns= ["Id"], inplace= True)

full_df.shape

(2917, 81)

In [98]:
# find categorical columns with missing values
c_missing = full_df.select_dtypes(include='object').isna().sum()
c_missing = c_missing[c_missing > 0].sort_values(ascending=False)
print(c_missing)

print("_"*100)

# find numeric columns with missing values
n_missing = full_df.select_dtypes(exclude='object').isna().sum()
n_missing = n_missing[n_missing > 0].sort_values(ascending=False)
print(n_missing)



PoolQC          2908
MiscFeature     2812
Alley           2719
Fence           2346
MasVnrType      1766
FireplaceQu     1420
GarageQual       159
GarageFinish     159
GarageCond       159
GarageType       157
BsmtCond          82
BsmtExposure      82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MSZoning           4
Functional         2
Utilities          2
Electrical         1
KitchenQual        1
Exterior2nd        1
Exterior1st        1
SaleType           1
dtype: int64
____________________________________________________________________________________________________
SalePrice       1459
LotFrontage      486
GarageYrBlt      159
MasVnrArea        23
BsmtFullBath       2
BsmtHalfBath       2
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
GarageCars         1
GarageArea         1
dtype: int64


In [99]:

 # using None for categorical columns, where it means NO/NONE
categorical_na_fill = [
    'PoolQC','MiscFeature','Alley','Fence','FireplaceQu',
    'GarageType','GarageFinish','GarageQual','GarageCond',
    'BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
    'MasVnrType']

for col in categorical_na_fill:
    if col in full_df.columns:
        full_df[col] = full_df[col].fillna('None')

#not including LotFrontage beacause its a meaninful continuous variable
# using 0 for numerical columns where NA means ZERO AREA/COUNTS
numerical_na_fill = ['GarageYrBlt','GarageArea','GarageCars',
                     'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF',
                     'BsmtFullBath','BsmtHalfBath','MasVnrArea']

for col in numerical_na_fill:
    if col in full_df.columns:
        full_df[col] = full_df[col].fillna(0)

#mode for the rest of categorical colums
for col in full_df.select_dtypes(include='object').columns:
    full_df[col] = full_df[col].fillna(full_df[col].mode()[0])

#median for the rest of numerical values
num_cols = full_df.select_dtypes(exclude='object').columns.tolist()
if 'SalePrice' in num_cols:
    num_cols.remove('SalePrice')
for col in num_cols:
    full_df[col] = full_df[col].fillna(full_df[col].median())


In [100]:
#Check to make sure the cleaning was done correctly 


print("categorical columns after cleaning:")

c_missing = full_df.select_dtypes(include='object').isna().sum()
c_missing = c_missing[c_missing > 0]

if c_missing.empty:
    print("No missing values in categorical columns")
else:
    print(cat_missing)

print("_"*100)

n_missing = full_df.drop(columns=['SalePrice'], errors='ignore') \
                     .select_dtypes(exclude='object') \
                     .isna().sum()

n_missing = n_missing[n_missing > 0]

print("Numerical missing values excluding SalePrice:")
print(n_missing if not n_missing.empty else "No missing values in the numerical colums")

categorical columns after cleaning:
No missing values in categorical columns
____________________________________________________________________________________________________
Numerical missing values excluding SalePrice:
No missing values in the numerical colums


In [101]:
#categorical codes disguised as numbers, will mess with the log-transform if not changed to object

print(full_df[['MSSubClass', 'MoSold', 'YrSold']].dtypes.head(3))

MSSubClass    int64
MoSold        int64
YrSold        int64
dtype: object


In [106]:
# changing from int64 to object for the log transform
for as_cat in ['MSSubClass','YrSold','MoSold']:
    if as_cat in full_df.columns:
        full_df[as_cat] = full_df[as_cat].astype(str)

full_df[['MSSubClass', 'MoSold', 'YrSold']].dtypes.head(15)

MSSubClass    object
MoSold        object
YrSold        object
dtype: object

In [107]:

# 1. Get all numeric features (ignore object/categorical ones)
n_features = full_df.select_dtypes(include=[np.number]).columns.tolist()

# 2. Remove SalePrice from the list (we only transform input features, not the target)
if 'SalePrice' in n_features:
    n_features.remove('SalePrice')

# 3. Calculate skewness for each numeric feature
skew_values = {}
for col in n_features:
    skew_values[col] = skew(full_df[col].dropna())

# 4. Find the features that are highly skewed (absolute skew > 0.75)
skewed_features = [col for col, value in skew_values.items() if abs(value) > 0.75]

print(f"we found {len(skewed_features)} skewed features. They will be log-transformed.")

# 5. Apply log1p transform to those skewed features
for col in skewed_features:
    full_df[col] = np.log1p(full_df[col])

we found 20 skewed features. They will be log-transformed.


In [108]:
full_df_encoded = pd.get_dummies(full_df, drop_first=False)
print('Encoded shape:', full_df_encoded.shape)
full_df_encoded.head(3)


Encoded shape: (2917, 334)


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,SalePrice,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,...,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_Fin,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,DatasetType_test,DatasetType_train
0,4.189655,9.042040,7,5,2003,2003,5.283204,6.561031,0.0,5.017280,856.0,6.753438,6.751101,0.0,7.444833,1.0,0.000000,2,1,3,0.693147,8,0,7.602900,2.0,548.0,0.000000,4.127134,0.0,0.0,0.0,0.0,0.0,208500.0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,

In [109]:
scaler = StandardScaler()

# Separate target before scaling
y_log = None
if 'SalePrice' in full_df_encoded.columns:
    y = full_df_encoded.loc[full_df_encoded['DatasetType_train'] == 1, 'SalePrice'].copy()
    y_log = np.log1p(y)  # log-transform target for modeling later
    X = full_df_encoded.drop(columns=['SalePrice'])
else:
    X = full_df_encoded

# Fit only on features
X_scaled = X.copy()
num_cols_scaled = X.select_dtypes(include=[np.number]).columns.tolist()
X_scaled[num_cols_scaled] = scaler.fit_transform(X[num_cols_scaled])

print('Scaled features shape:', X_scaled.shape)
X_scaled.head(3)

Scaled features shape: (2917, 333)


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,...,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_Fin,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,DatasetType_test,DatasetType_train
0,-0.037832,-0.101761,0.649580,-0.507503,1.047231,0.897702,1.224732,0.784689,-0.362706,-0.324876,-0.449391,-0.778480,1.198058,-0.116973,0.570217,1.090257,-0.251636,0.783237,1.233664,0.169987,-0.200585,0.992279,-0.924763,0.247405,0.307738,0.352788,-0.944441,0.834573,-0.427719,-0.112813,-0.30926,-0.064082,-0.189396,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,Fals

In [111]:
# Assume X_scaled is your scaled DataFrame after preprocessing
# It still has the helper column DatasetType_train to mark rows

# Separate train and test sets
train_clean = X_scaled.loc[X_scaled['DatasetType_train'] == 1].copy()
test_clean  = X_scaled.loc[X_scaled['DatasetType_train'] == 0].copy()

# Add target SalePrice back to train (remember we saved y_log or y earlier)
train_clean['SalePrice'] = y_log  # or y if not log-transformed

# Drop helper columns if you don’t want them in saved files
train_clean = train_clean.drop(columns=['DatasetType_train'], errors='ignore')
test_clean  = test_clean.drop(columns=['DatasetType_train'], errors='ignore')

# Save to CSV
train_clean.to_csv("train_clean.csv", index=False)
test_clean.to_csv("test_clean.csv", index=False)

print("Saved train_clean.csv and test_clean.csv")


Saved train_clean.csv and test_clean.csv


In [112]:
print("Train shape:", train_clean.shape)
print("Test shape :", test_clean.shape)

Train shape: (1458, 333)
Test shape : (1459, 332)


In [115]:
train_df = pd.read_csv("train_clean.csv") 
test_df  = pd.read_csv("test_clean.csv")

print(train_df.shape, test_df.shape)
display(train_df.head(10)) 


(1458, 333) (1459, 332)


,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_None,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,...,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_None,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,GarageFinish_Fin,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_1,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,DatasetType_test,SalePrice
0,-0.037832,-0.101761,0.649580,-0.507503,1.047231,0.897702,1.224732,0.784689,-0.362706,-0.324876,-0.449391,-0.778480,1.198058,-0.116973,0.570217,1.090257,-0.251636,0.783237,1.233664,0.169987,-0.200585,0.992279,-0.924763,0.247405,0.307738,0.352788,-0.944441,0.834573,-0.427719,-0.112813,-0.30926,-0.064082,-0.189396,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,False,True,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,True,False,True,False,False,False,

In [116]:

# Optional: ensure xgboost is available (uncomment the next line if needed)
%pip install xgboost -q

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
try:
    from xgboost import XGBRegressor
    xgb_available = True
except Exception as e:
    print("⚠️ XGBoost not available. You can install it with `%pip install xgboost`.")
    xgb_available = False

# Load cleaned data produced earlier
train_df = pd.read_csv("train_clean.csv")
test_df  = pd.read_csv("test_clean.csv")


# Features & target
X = train_df.drop(columns=["SalePrice"])
y = train_df["SalePrice"]

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print('Shapes:')
print('  X_train:', X_train.shape, 'X_val:', X_val.shape)

Note: you may need to restart the kernel to use updated packages.
Shapes:
  X_train: (1166, 332) X_val: (292, 332)


In [118]:

# 1) Linear Regression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# 2) Random Forest
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# 3) XGBoost 
if xgb_available:
    xgb = XGBRegressor(
        n_estimators=2000, learning_rate=0.05, max_depth=3,
        subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1)
else:
    xgb = None


In [119]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

def evaluate_model(name, model, X_train, X_val, y_train, y_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mae  = mean_absolute_error(y_val, y_pred)
    r2   = r2_score(y_val, y_pred)

    return {"Model": name, "RMSE": rmse, "MAE": mae, "R²": r2}

results = []

results.append(evaluate_model("Linear Regression", linreg, X_train, X_val, y_train, y_val))
results.append(evaluate_model("Random Forest", rf, X_train, X_val, y_train, y_val))
results.append(evaluate_model("XGBoost", xgb, X_train, X_val, y_train, y_val))

results_df = pd.DataFrame(results).sort_values("RMSE").reset_index(drop=True)
display(results_df)


,Model,RMSE,MAE,R²
0,XGBoost,1.178431e-01,8.288563e-02,9.176219e-01
1,Random Forest,1.438955e-01,9.701967e-02,8.771718e-01
2,Linear Regression,1.164190e+10,1.200856e+09,-8.039908e+20


In [121]:

best_row = results_df.iloc[0]
best_name = best_row['Model']
print(f'Best model on validation: {best_name} (RMSE={best_row.RMSE:.4f})')

# Instantiate same hyperparameters for the best model
if best_name == 'LinearRegression':
    best_model = LinearRegression()
elif best_name == 'RandomForest':
    best_model = RandomForestRegressor()
elif best_name == 'XGBoost':
    best_model = XGBRegressor()
else:
    raise ValueError('Unexpected model name')

# Retrain on the full training set
best_model.fit(X, y)
print('Best model retrained on all training data.')

Best model on validation: XGBoost (RMSE=0.1178)
Best model retrained on all training data.


In [133]:
import pandas as pd
import numpy as np


train_clean = pd.read_csv("train_clean.csv")
test_clean  = pd.read_csv("test_clean.csv")


X_all = train_clean.drop(columns=["SalePrice"])
y_all = train_clean["SalePrice"]

# Keep original Ids for submission
test_ids = pd.read_csv("test.csv")["Id"].values


best_model.fit(X_all, y_all)
print(f"Retrained final model: {best_name}")

#back to dollar
test_pred_log = best_model.predict(test_clean)   
test_pred = np.expm1(test_pred_log)              


submission = pd.DataFrame({
    "Id": test_ids,
    "SalePrice": test_pred
})



submission.to_csv("submission.csv", index=False)
print("Saved submission.csv")

# Preview
display(submission.head())


Retrained final model: XGBoost
Saved submission.csv


,Id,SalePrice
0,1461,119776.921875
1,1462,161093.296875
2,1463,184725.171875
3,1464,188625.625000
4,1465,183591.312500
